<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/CIFAR10_AUC_ACC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/XuZhengzhuo/Prior-LT.git
%cd Prior-LT

Cloning into 'Prior-LT'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 54 (delta 1), reused 0 (delta 0), pack-reused 48
Unpacking objects: 100% (54/54), done.
/content/Prior-LT


In [2]:
#src: https://github.com/XuZhengzhuo/Prior-LT
import gdown
url = 'https://drive.google.com/uc?id=1tclscVkcXj0lJum7Azy8qHecB7Pomc0c'
gdown.download(url,'model_best_cifar10_lt50.pth.tar',quiet=False) 

Downloading...
From: https://drive.google.com/uc?id=1tclscVkcXj0lJum7Azy8qHecB7Pomc0c
To: /content/Prior-LT/model_best_cifar10_lt50.pth.tar
100%|██████████| 3.77M/3.77M [00:00<00:00, 203MB/s]


'model_best_cifar10_lt50.pth.tar'

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import _LRScheduler
import matplotlib.pyplot as plt
from PIL import Image
import copy 
import os
import argparse
import sys
import random
import numpy as np
from torchvision import models
from models import resnet32
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total
def main(num_classes=100, ckpt=None, test_loader=None):
    model = resnet32(num_classes=num_classes)
    model.load_state_dict(torch.load(ckpt)['state_dict_model'])
    model.to(device)
    acc = test(model, test_loader)
    return acc

mean_cifar, std_cifar = (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
transform_test = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize(mean_cifar, std_cifar),])

test_dataset10 = torchvision.datasets.CIFAR10(root='data', train=False, download=True, transform=transform_test)
test_loader10 = torch.utils.data.DataLoader(test_dataset10, batch_size=2048, shuffle=False, num_workers=2)

main(num_classes=10, ckpt='model_best_cifar10_lt50.pth.tar', test_loader=test_loader10)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data


0.8493

#AUC Score for multi-class classification
src: https://thomasjpfan.github.io/scikit-learn-website/modules/generated/sklearn.metrics.roc_auc_score.html<br>

multi_classstring, ‘ovr’ or ‘ovo’, optional(default=’raise’)
Determines the type of multiclass configuration to use. multi_class must be provided when y_true is multiclass.

'ovr':
Calculate metrics for the multiclass case using the one-vs-rest approach.

'ovo':
Calculate metrics for the multiclass case using the one-vs-one approach.

In [25]:
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    probability_all = []
    targets_all = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            logits = model(inputs)
            probability = F.softmax(logits, dim=1)
            probability_all.extend(probability.cpu().numpy())
            targets_all.extend(targets.cpu().numpy())
            _, predicted = logits.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total, np.array(probability_all), np.array(targets_all)

model = resnet32(num_classes=10)
model.load_state_dict(torch.load('model_best_cifar10_lt50.pth.tar')['state_dict_model'])
model.to(device)
acc, probability_all, targets_all  = test(model, test_loader10)
auc = roc_auc_score( targets_all, probability_all, multi_class='ovo')

print('CIFAR10: accuracy:', acc, 'auc:',auc)


CIFAR10: accuracy: 0.8493 auc: 0.9852102444444445
